<a href="https://colab.research.google.com/github/Aditi0003/IDC409_Project-/blob/main/DS_project_HEP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center>${\text{IDC 409 - Introduction to Data Science}}$</center>
# <center>${\text{Project 02: HEP Event Classification}}$</center>

# <center>${\text{Group Members:}}$</center>
#### <center>${\text{Kunal Kashyap (MS20030)}}$</center>
#### <center>${\text{Aditi (MS20020)}}$</center>
#### <center>${\text{Dhwani Shah (MS20156)}}$</center>

## Loading basic packages and mounting drive

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Loading the Data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data_hep.csv")
# Removing the first column (that is index)
df = df.drop("Unnamed: 0",axis=1)

In [ ]:
# Visualizing the data
df

,R2,thrustBm,thrustOm,cosTBTO,cosTBz,KSFWVariables__boet__bc,KSFWVariables__bomm2__bc,CleoConeCS__bo1__bc,CleoConeCS__bo2__bc,CleoConeCS__bo3__bc,...,KSFWVariables__bohoo2__cmFS1__bc,KSFWVariables__bohoo3__cmFS1__bc,KSFWVariables__bohoo4__cmFS1__bc,foxWolframR1,foxWolframR2,foxWolframR3,foxWolframR4,thrust,thrustAxisCosTheta,type
0,0.152200,0.818042,0.678908,0.925376,0.641596,5.492738,7.326161,0.062815,0.923991,1.262607,...,-0.000648,-0.000956,0.003297,0.315513,0.257067,0.165595,0.099685,0.766103,0.584374,1
1,0.122531,0.847877,0.661546,0.203175,0.960107,3.548602,5.346302,0.000000,2.317477,0.000000,...,-0.004079,0.000975,-0.010682,0.699070,0.425364,0.337010,0.357491,0.842736,0.960207,1
2,0.071109,0.823682,0.659284,0.586194,0.667930,7.739933,-1.734496,0.393218,0.061572,2.128880,...,-0.002341,-0.002249,-0.000644,0.351783,0.278459,0.247533,0.171277,0.773620,0.723453,1
3,0.123307,0.849041,0.713995,0.343100,0.599684,6.336428,5.429208,0.000000,0.000000,2.233941,...,-0.001151,-0.000096,0.003249,0.616451,0.581567,0.504908,0.313320,0.906921,0.631777,1
4,0.128661,0.850345,0.700636,0.598284,0.751411,7.520575,-1.092915,0.105494,0.789670,2.863936,...,0.004459,-0.001671,-0.000072,0.099653,0.368981,0.190910,0.247606,0.824495,0.589225,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70601,0.341170,0.822778,0.905524,0.744942,0.883310,5.254729,2.711209,0.229089,0.000000,0.411111,...,0.007728,-0.003148,0.013386,0.240637,0.320524,0.114465,0.172375,0.789952,0.865978,2
70602,0.431197,0.890331,0.851992,0.979008,0.270003,10.016900,-1.206864,0.000000,4.097687,2.356474,...,0.063099,-0.006214,0.009684,0.254839,0.436554,0.169985,0.252282,0.861856,-0.084869,2
70603,0.345012,0.897968,0.745580,0.974656,0.691979,8.154435,-0.880221,4.716620,1.190169,1.030242,...,0.026411,0.000968,0.004867,0.089218,0.603980,0.223313,0.441302,0.903512,0.689248,2
70604,0.240343,0.910267,0.698762,0.960566,0.441567,9.145159,-9.401502,2.592608,0.548217,3.795009,...,0.020226,-0.006749,0.021568,0.406443,0.524733,0.335748,0.384065,0.868288,-0.230441,2


In [ ]:
# Obtaining X nd Y (Labels) from the dataframe.
Y = df["type"].values
X = df.drop("type",axis=1).values
# Hot encoding Y values (1 in ith index if it is a i+1 class event)
Y=np.eye(5)[Y-1]

# Approach 1:
\\
Use a simple sequential neural network (MLP) to classify data without dimension reduction

In [ ]:
# Importing required tenserflow keras packages for MLP construction
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout
# Splitting the data into training and validation subsets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Standardize the input data (IMPORTANT)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture (Using simple MLP)
model = Sequential()
model.add(Dense(625, input_dim= 59))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(225))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Dense(5, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
history = model.fit(X_train, Y_train, epochs=50, batch_size=500, validation_data=(X_test, Y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')



Epoch 1/50
113/113 [==============================] - 3s 8ms/step - loss: 1.0654 - accuracy: 0.6403 - val_loss: 0.8899 - val_accuracy: 0.6992
Epoch 2/50
113/113 [==============================] - 1s 6ms/step - loss: 0.8354 - accuracy: 0.7048 - val_loss: 0.7974 - val_accuracy: 0.7121
Epoch 3/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7887 - accuracy: 0.7147 - val_loss: 0.7742 - val_accuracy: 0.7217
Epoch 4/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7688 - accuracy: 0.7201 - val_loss: 0.7624 - val_accuracy: 0.7243
Epoch 5/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7576 - accuracy: 0.7249 - val_loss: 0.7541 - val_accuracy: 0.7299
Epoch 6/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7500 - accuracy: 0.7276 - val_loss: 0.7476 - val_accuracy: 0.7302
Epoch 7/50
113/113 [==============================] - 1s 8ms/step - loss: 0.7419 - accuracy: 0.7312 - val_loss: 0.7418 - val_accuracy: 0.7330
Epoch 

<b>We could obtain an accuracy ~ 75% using this method</b>

# Approach 2:
\\
Use PCA to reduce dimentionality and then use a simple sequential neural network (MLP) to classify data

In [ ]:
# Required packages
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Dropout, ReLU
from sklearn.decomposition import PCA # PAckage for PCA



# Standardizing the input data
scaler = StandardScaler()
X_T = scaler.fit_transform(X)


#PCA to reduce dimentionality
pca = PCA(n_components = 20)  #Keeping number of components = 20
pca.fit(X_T)
X_P = pca.fit_transform(X_T)



# Obtaibing train and Test subsets
X_train, X_test, Y_train, Y_test = train_test_split(X_P, Y, test_size=0.2, random_state=42)




# Define the neural network architecture
model = Sequential()
model.add(Dense(625, input_dim= 20))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(225))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(20))
model.add(Dense(5, activation='softmax'))


# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, epochs=20, batch_size=500, validation_data=(X_test, Y_test))

# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')


Epoch 1/20
113/113 [==============================] - 4s 13ms/step - loss: 1.0569 - accuracy: 0.6639 - val_loss: 0.9079 - val_accuracy: 0.7011
Epoch 2/20
113/113 [==============================] - 1s 10ms/step - loss: 0.8402 - accuracy: 0.7003 - val_loss: 0.8538 - val_accuracy: 0.6971
Epoch 3/20
113/113 [==============================] - 1s 11ms/step - loss: 0.8002 - accuracy: 0.7094 - val_loss: 0.8204 - val_accuracy: 0.7128
Epoch 4/20
113/113 [==============================] - 1s 9ms/step - loss: 0.7820 - accuracy: 0.7149 - val_loss: 0.7857 - val_accuracy: 0.7109
Epoch 5/20
113/113 [==============================] - 1s 8ms/step - loss: 0.7737 - accuracy: 0.7166 - val_loss: 0.7782 - val_accuracy: 0.7163
Epoch 6/20
113/113 [==============================] - 1s 10ms/step - loss: 0.7656 - accuracy: 0.7175 - val_loss: 0.7729 - val_accuracy: 0.7229
Epoch 7/20
113/113 [==============================] - 1s 9ms/step - loss: 0.7578 - accuracy: 0.7211 - val_loss: 0.7777 - val_accuracy: 0.7178
Ep

<b>We could obtain accuracy ~71% with dimensionality reduced to 20. In general, the accuracy decreased slowly as we reduced the number of components</b>